In [1]:
#| default_exp core

# OSOb

> This is a python module providing communication abstractions for the OpenScience Observatories telescope.org service. The telescope control is provided through the `Telescope` class which provides state tracking and low level methods - forming a basic API layer. The higher level functions are implemented as separate functions construced with the Telescope class API.

In [2]:
#| hide
from nbdev.showdoc import *
from nbdev import *

In [3]:
#| export
import logging
from requests import session
from bs4 import BeautifulSoup
import configparser
from os.path import expanduser
import json
from fastcore.basics import patch
from zipfile import ZipFile, BadZipFile
from tqdm.auto import tqdm
import sys
import os
import time
from astropy.coordinates import SkyCoord, Longitude, Latitude

## Telescope class

This class orginizes all interaction with the service. It keeps all state and provides higher level functions like `login` or `get_user_requests`.

In [4]:
#| export
class Telescope :

    url='https://www.telescope.org/'
    cameratypes={
        'constellation':'1',
        'galaxy':       '2',
        'cluster':      '3',
        'planet':'5',
        'coast':'6',
        'pirate':'7',
    }

    REQUESTSTATUS_TEXTS={
        1: "New",
        2: "New, allocated",
        3: "Waiting",
        4: "In progress",
        5: "Reallocate",
        6: "Waiting again",
        7: "Complete on site",
        8: "Complete",
        9: "Hold",
        10: "Frozen",
        20: "Expired",
        21: "Expired w/CJobs",
        22: "Cancelled",
        23: "Cancelled w/CJobs",
        24: "Invalid",
        25: "Never rises",
        26: "Other error",
    }

    def __init__(self,user,passwd,cache='.cache/jobs'):
        self.s=None
        self.user=user
        self.passwd=passwd
        self.tout=60
        self.retry=15
        self.login()
        self.cache=cache

In [5]:
#| export
def cleanup(s):
    return s.encode('ascii','ignore').decode('ascii','ignore')

In [6]:
assert cleanup('|ążźćńłóęśĄŻŹĆŃŁÓĘŚ|') == '||'

In [7]:
#| export
@patch
def login(self: Telescope):
    log = logging.getLogger(__name__)
    payload = {'action': 'login',
               'username': self.user,
               'password': self.passwd,
               'stayloggedin': 'true'}
    log.debug('Get session ...')
    self.s=session()
    log.debug('Logging in ...')
    self.s.post(self.url+'login.php', data=payload)

In [8]:
#| export
@patch
def logout(self: Telescope):
    if self.s is None :
        self.s.post(self.url+'logout.php')
        self.s=None
 

In [9]:
#| local
config = configparser.ConfigParser()
config.read(expanduser('~/.config/telescope.ini'))

['/home/jochym/.config/telescope.ini']

In [10]:
#| local
OSO=Telescope(config['telescope.org']['user'], 
              config['telescope.org']['password'])

In [11]:
#| export
@patch
def get_user_requests(self: Telescope, sort='rid', folder=1):
    '''
    Get all user requests from folder (Inbox=1 by default),
    sorted by sort column ('rid' by default). 
    Possible sort columns are: 'rid', 'object', 'completion'
    The data is returned as a list of dictionaries.
    '''

    #fetch first batch        
    params={
        'limit': 100,
        'sort': sort,
        'folderid': folder}

    rq = self.s.post(self.url+"api-user.php", {'module': "request-manager", 
                                               'request': "1-get-list-own",
                                               'params' : json.dumps(params)})
    res=[]
    dat=json.loads(rq.content)
    total=int(dat['data']['totalRequests'])
    res+=dat['data']['requests']

    # Fetch the rest
    params['limit']=total-len(res)
    params['startAfterRow']=len(res)
    rq = self.s.post(self.url+"api-user.php", {'module': "request-manager", 
                                               'request': "1-get-list-own",
                                               'params' : json.dumps(params)})

    dat=json.loads(rq.content)
    total=int(dat['data']['totalRequests'])
    res+=dat['data']['requests']
    return res

In [12]:
#| local
reqlst=OSO.get_user_requests(sort='completion')
print(f'Number of users requests: {len(reqlst)}')
reqlst[0]

Number of users requests: 1189


{'id': '725917',
 'seen': '0',
 'usercomments': '',
 'objecttype': 'RADEC',
 'objectid': '18:55:02.31 -31:09:49.59',
 'objectname': 'V1223 Sgr',
 'requesttime': '1630285858',
 'status': '3',
 'row': '1'}

In [13]:
#| export
@patch
def get_request(self: Telescope, rid=None):
    '''Get request data for a given RID'''

    assert(rid is not None)
    assert(self.s is not None)

    log = logging.getLogger(__name__)
    log.debug(rid)

    obs={}
    obs['rid']=rid
    #rq=self.s.post(self.url+('v3cjob-view.php?jid=%d' % jid))
    rq=self.s.post(self.url+('v4request-view.php?rid=%d' % rid))
    soup = BeautifulSoup(rq.text, 'lxml')
    for l in soup.findAll('tr'):
        log.debug(cleanup(l.text))
        txt=''
        for f in l.findAll('td'):
            if txt.find('Job ID') >= 0:
                obs['jid']=f.text
            if txt.find('Object Type') >= 0:
                obs['type']=f.text
            if txt.find('Object ID') >= 0:
                obs['oid']=f.text
            if txt.find('Object Name') >= 0:
                obs['name']=f.text
            if txt.find('Telescope Type Name') >= 0:
                obs['tele_type']=f.text
            if txt.find('Telescope Name') >= 0:
                obs['tele']=f.text
            if txt.find('Filter Type') >= 0:
                obs['filter']=f.text
            if txt.find('Dark Frame') >= 0:
                obs['dark']=f.text
            if txt.find('Exposure Time') >= 0:
                obs['exp']=f.text
            if txt.find('Request Time') >= 0:
                t=f.text.split()
                obs['requested']=t[3:6]+[t[6][1:]]+[t[7][:-1]]
            if txt.find('Completion Time') >= 0:
                t=f.text.split()
                obs['completion']=t[3:6]+[t[6][1:]]+[t[7][:-1]]
            if txt.find('Status') >= 0:
                obs['status']= f.text.strip() #(f.text == 'Success')

            txt=f.text
    for l in soup.findAll('a'):
        if l.get('href')is not None and ('dl-flat' in l.get('href')):
            obs['flatid']=int(l.get('href').split('=')[1])
            break
    log.info('%(jid)d [%(tele)s, %(filter)s, %(status)s]: %(type)s %(oid)s %(exp)s', obs)

    return obs    

In [14]:
#| export
@patch
def get_user_folders(self: Telescope):
    '''
    Get all user folders. Returns list of dictionaries.
    '''
    rq = self.s.post(self.url+"api-user.php", {'module': "request-manager", 
                                               'request': "0-get-my-folders"})
    return json.loads(rq.content)['data']

In [15]:
#| local
OSO.get_user_folders()

[{'id': '1', 'creationtime': '0', 'name': 'Inbox', 'count': '1189'},
 {'id': '2', 'creationtime': '0', 'name': 'Favourites', 'count': None},
 {'id': '3', 'creationtime': '0', 'name': 'Archive', 'count': '447'},
 {'id': '4', 'creationtime': '0', 'name': 'Trash', 'count': '52'},
 {'id': '461',
  'creationtime': '1407254495',
  'name': 'Complete',
  'count': '13'}]

In [16]:
#| export
@patch
def get_obs_list(self: Telescope, t=None, dt=1, filtertype='', camera='', hour=16, minute=0):
    '''Get the dt days of observations taken no later then time in t.

        Input
        ------
        t  - end time in seconds from the epoch
            (as returned by time.time())
        dt - number of days, default to 1
        filtertype - filter by type of filter used
        camera - filter by the camera/telescope used

        Output
        ------
        Returns a list of JobIDs (int) for the observations.

    '''

    assert(self.s is not None)

    if t is None :
        t=time.time()-time.timezone


    st=time.gmtime(t-86400*dt)
    et=time.gmtime(t)

    d=st.tm_mday
    m=st.tm_mon
    y=st.tm_year
    de=et.tm_mday
    me=et.tm_mon
    ye=et.tm_year

    log = logging.getLogger(__name__)
    log.debug('%d/%d/%d -> %d/%d/%d', d,m,y,de,me,ye)

    try :
        telescope=self.cameratypes[camera.lower()]
    except KeyError:
        telescope=''

    searchdat = {
        'sort1':'completetime',
        'sort1order':'desc',
        'searchearliestcom[]':[d, m, y, str(hour),str(minute)],
        'searchlatestcom[]':  [de,me,ye,str(hour),str(minute)],
        'searchstatus[]':['1'],
        'resultsperpage':'1000',
        'searchfilter':filtertype,
        'searchtelescope':telescope,
        'submit':'Go'
    }

    headers = {'Content-Type': 'application/x-www-form-urlencoded'}


    request = self.s.post(self.url+'v3job-search-query.php',
                     data=searchdat, headers=headers)
    soup = BeautifulSoup(request.text,'lxml')

    jlst=[]
    for l in soup.findAll('tr'):
        try :
            a=l.find('a').get('href')
        except AttributeError :
            continue
        jid=a.rfind('jid')
        if jid>0 :
            jid=a[jid+4:].split('&')[0]
            jlst.append(int(jid))
    return jlst


In [17]:
#| local 
import datetime

olst = OSO.get_obs_list(t=datetime.datetime(2020, 12, 24).timestamp())
print(f'Observations: {len(olst)}')

Observations: 54


In [18]:
#| export
@patch
def get_job(self: Telescope, jid=None):
    '''Get a job data for a given JID'''

    assert(jid is not None)
    assert(self.s is not None)

    log = logging.getLogger(__name__)
    log.debug(jid)

    obs={}
    obs['jid']=jid
    #rq=self.s.post(self.url+('v3cjob-view.php?jid=%d' % jid))
    rq=self.s.post(self.url+('v4request-view.php?jid=%d' % jid))
    soup = BeautifulSoup(rq.text, 'lxml')
    for l in soup.findAll('tr'):
        log.debug(cleanup(l.text))
        txt=''
        for f in l.findAll('td'):
            if txt.find('Request ID') >= 0:
                obs['rid']=f.text            
            if txt.find('Object Type') >= 0:
                obs['type']=f.text
            if txt.find('Object ID') >= 0:
                obs['oid']=f.text
            if txt.find('Telescope Type Name') >= 0:
                obs['tele']=f.text
            if txt.find('Filter Type') >= 0:
                obs['filter']=f.text
            if txt.find('Exposure Time') >= 0:
                obs['exp']=f.text
            if txt.find('Completion Time') >= 0:
                t=f.text.split()
                obs['completion']=t[3:6]+[t[6][1:]]+[t[7][:-1]]
            if txt.find('Status') >= 0:
                obs['status']= (f.text == 'Success')

            txt=f.text
    for l in soup.findAll('a'):
        if l.get('href')is not None and ('dl-flat' in l.get('href')):
            obs['flatid']=int(l.get('href').split('=')[1])
            break
    log.info('%(jid)d [%(tele)s, %(filter)s, %(status)s]: %(type)s %(oid)s %(exp)s', obs)

    return obs

In [19]:
#| local
for rq in sorted(reqlst, key=lambda r: int(r['requesttime']), reverse=True):
    if Telescope.REQUESTSTATUS_TEXTS[int(rq['status'])]=='Complete':
        break
print(rq)
print(OSO.get_request(int(rq['id'])))
last_complete = int(OSO.get_request(int(rq['id']))['jid'][1:])

{'id': '725594', 'seen': '1', 'usercomments': 'Mira', 'objecttype': 'RADEC', 'objectid': '20:00:03.02 +22:46:51.55', 'objectname': 'DQ Vul', 'requesttime': '1629985074', 'status': '8', 'row': '6'}
{'rid': 725594, 'jid': 'J383652', 'type': 'RADEC', 'oid': '20:00:03.02 +22:46:51.55', 'name': 'DQ Vul', 'exp': '180000 ms', 'filter': 'BVR', 'dark': 'Instant', 'tele_type': 'Galaxy', 'tele': 'COAST', 'requested': ['26', 'August', '2021', '13:37:54', 'UTC'], 'completion': ['31', 'August', '2021', '02:51:11', 'UTC'], 'status': 'Complete', 'flatid': 26}


### Basic API calls

> User-API

> Request Manager

> Request Constructor

In [20]:
#| export
@patch
def do_api_call(self: Telescope, module, req, params=None):
    rq = self.s.post(self.url+"api-user.php", {'module': module,
                                               'request': req,
                                               'params': {} if params is None else json.dumps(params)})
    return json.loads(rq.content)

In [21]:
#| local
obs = OSO.get_job(last_complete)
rsp = OSO.do_api_call("image-engine", "0-create-dlzip", {'jid': obs['jid'], 'flatid': obs['flatid']})
print(rsp)
rsp = OSO.do_api_call("image-engine", "0-is-job-ready", {'ieid':rsp['data']['ieID'],})
print(rsp)

{'success': 1, 'status': 'OK_WAIT', 'data': {'ieID': '1550470'}}
{'success': 1, 'status': 'PROCESSING', 'data': None}


In [22]:
#| export
@patch
def do_rm_api(self: Telescope, req, params=None):
    return self.do_api_call("request-manager", req, params)


#export
@patch
def do_rc_api(self: Telescope, req, params=None):
    return self.do_api_call("request-constructor", req, params)

In [23]:
#| export
@patch
def download_obs(self: Telescope, obs=None, directory='.', cube=True, verbose=False):
    '''Download the raw observation obs (obtained from get_job) into zip
    file named job_jid.zip located in the directory (current by default).
    Alternatively, when the cube=True the file will be a 3D fits file.
    The name of the file (without directory) is returned.'''

    assert(obs is not None)
    assert(self.s is not None)

    payload = {'jid': obs['jid']}
    if 'flatid' in obs :
        payload['flatid']=obs['flatid']
    
    rsp = self.do_api_call("image-engine", 
                           "0-create-dl" + ("3d" if cube else "zip"), payload)
    ieid = rsp['data']['ieID']

    n=0
    while rsp['status']!='READY' :
        if verbose:
            print(f"{rsp['status']:30}", end='\n')
        time.sleep(2)
        n+=1
        rsp = self.do_api_call("image-engine", "0-is-job-ready", {'ieid':ieid,})
        if n>30:
            raise TimeoutError
    
    if verbose:
        print(f"{rsp['status']:30}")
        sys.stdout.flush()
    
    rq=self.s.get(self.url+f'v3image-download.php?jid={obs["jid"]}&ieid={ieid}', 
                  stream=True)

    fn = ('%(jid)d.' % obs) + ('fits' if cube else 'zip')
    siz = int(rsp['data']['fitssize' if cube else 'fitsbzsize'])
    pbar = tqdm(total=siz, unit='iB', unit_scale=True, disable=not verbose)
    with open(os.path.join(directory, fn), 'wb') as fd:
        for chunk in rq.iter_content(512):
            pbar.update(len(chunk))
            fd.write(chunk)
    pbar.close()
    sys.stdout.flush()
    if siz==os.stat(os.path.join(directory, fn)).st_size :
        return fn
    else:
        return None

In [24]:
#| local
fn = OSO.download_obs(OSO.get_job(last_complete), directory='/tmp', cube=False, verbose=True)
if fn is not None:
    print(f'Removing downloaded file: {fn}')
    os.unlink(os.path.join('/tmp', fn))
else:
    print('Download failed')

OK_WAIT                       
WAIT                          
WAIT                          
READY                         


  0%|          | 0.00/9.01M [00:00<?, ?iB/s]

Removing downloaded file: 383652.zip


In [25]:
#| local
fn = OSO.download_obs(OSO.get_job(last_complete), directory='/tmp', cube=True, verbose=True)
if fn is not None:
    print(f'Removing downloaded file: {fn}')
    os.unlink(os.path.join('/tmp', fn))
else:
    print('Download failed')

OK_WAIT                       
READY                         


  0%|          | 0.00/14.2M [00:00<?, ?iB/s]

Removing downloaded file: 383652.fits


In [26]:
#| export
@patch
def get_obs(self: Telescope, obs=None, cube=True, recurse=True, verbose=False):
    '''Get the raw observation obs (obtained from get_job) into zip
    file-like object. The function returns ZipFile structure of the
    downloaded data.'''

    assert(obs is not None)
    assert(self.s is not None)

    log = logging.getLogger(__name__)

    fn = ('%(jid)d.' % obs) + ('fits' if cube else 'zip')
    fp = os.path.join(self.cache,fn[0],fn[1],fn)
    if not os.path.isfile(fp) :
        log.info('Getting %s from server', fp)
        os.makedirs(os.path.dirname(fp), exist_ok=True)
        self.download_obs(obs, os.path.dirname(fp), cube, verbose)
    else :
        log.info('Getting %s from cache', fp)
    content = open(fp,'rb')
    try :
        return content if cube else ZipFile(content)
    except BadZipFile :
        # Probably corrupted download. Try again once.
        content.close()
        os.remove(fp)
        if recurse :
            return self.get_obs(obs, cube, False, verbose)
        else :
            return None

In [27]:
#| local
(OSO.get_obs(OSO.get_job(last_complete), cube=False, verbose=True), 
OSO.get_obs(OSO.get_job(last_complete), cube=True, verbose=True),)

OK_READY                      
READY                         


  0%|          | 0.00/14.2M [00:00<?, ?iB/s]

(<zipfile.ZipFile file=<_io.BufferedReader name='.cache/jobs/3/8/383652.zip'> mode='r'>,
 <_io.BufferedReader name='.cache/jobs/3/8/383652.fits'>)

### Job submission methods

> Submission API

In [28]:
#| export
@patch
def submit_job_api(self: Telescope, obj, exposure=30000, tele='COAST',
                    filt='BVR', darkframe=True,
                    name='RaDec object', comment='AutoSubmit'):
    assert(self.s is not None)

    log = logging.getLogger(__name__)

    ra=obj.ra.to_string(unit='hour', sep=':', pad=True, precision=2,
                        alwayssign=False)
    dec=obj.dec.to_string(sep=':', pad=True, precision=2,
                        alwayssign=True)
    try :
        tele=self.cameratypes[tele.lower()]
    except KeyError :
        log.warning('Wrong telescope: %d ; selecting COAST(6)', tele)
        tele=6

    if tele==7 :
        if filt=='BVR' : filt='Colour'
        if filt=='B' : filt='Blue'
        if filt=='V' : filt='Green'
        if filt=='R' : filt='Red'
    if tele==6 :
        if filt=='Colour' : filt='BVR'
        if filt=='Blue' : filt='B'
        if filt=='Green' : filt='V'
        if filt=='Red' : filt='R'

    params = {'telescopeid': tele, 'telescopetype': 2,
              'exposuretime': exposure, 'filtertype': filt,
              'objecttype': 'RADEC', 'objectname': name,
              'objectid': ra+' '+dec, 'usercomments': comment }

    self.do_rc_api("0-rb-clear")

    r = self.do_rc_api("0-rb-set", params)
    log.debug('Req data:%s', r)
    if r['success'] :
        r = self.do_rc_api("0-rb-submit")
        log.debug('Submission data:%s', r)
    if r['success'] :
        return True, r['data']['id']
    else :
        log.warning('Submission error. Status:%s', r['status'])
        return False, r['status']

### RADEC job submission

In [29]:
#| export
@patch
def submit_RADEC_job(self: Telescope, obj, exposure=30000, tele='COAST',
                    filt='BVR', darkframe=True,
                    name='RaDec object', comment='AutoSubmit'):
    assert(self.s is not None)

    log = logging.getLogger(__name__)

    ra=obj.ra.to_string(unit='hour', sep=' ',
                        pad=True, precision=2,
                        alwayssign=False).split()
    dec=obj.dec.to_string(sep=' ',
                        pad=True, precision=2,
                        alwayssign=True).split()
    try :
        tele=self.cameratypes[tele.lower()]
    except KeyError :
        log.warning('Wrong telescope: %d ; selecting COAST(6)', tele)
        tele=6

    if tele==7 :
        if filt=='BVR' : filt='Colour'
        if filt=='B' : filt='Blue'
        if filt=='V' : filt='Green'
        if filt=='R' : filt='Red'
    if tele==6 :
        if filt=='Colour' : filt='BVR'
        if filt=='Blue' : filt='B'
        if filt=='Green' : filt='V'
        if filt=='Red' : filt='R'

    u=self.url+'/request-constructor.php'
    r=self.s.get(u+'?action=new')
    t=self.extract_ticket(r)
    log.debug('GoTo Part 1 (ticket %s)', t)
    r=self.s.post(u,data={'ticket':t,'action':'main-go-part1'})
    t=self.extract_ticket(r)
    log.debug('GoTo RADEC (ticket %s)', t)
    r=self.s.post(u,data={'ticket':t,'action':'part1-go-radec'})
    t=self.extract_ticket(r)
    log.debug('Save RADEC (ticket %s)', t)
    r=self.s.post(u,data={'ticket':t,'action':'part1-radec-save',
                         'raHours':ra[0],
                         'raMins':ra[1],
                         'raSecs':ra[2].split('.')[0],
                         'raFract':ra[2].split('.')[1],
                         'decDegrees':dec[0],
                         'decMins':dec[1],
                         'decSecs':dec[2].split('.')[0],
                         'decFract':dec[2].split('.')[1],
                         'newObjectName':name})
    t=self.extract_ticket(r)
    log.debug('GoTo Part 2 (ticket %s)', t)
    r=self.s.post(u,data={'ticket':t,'action':'main-go-part2'})
    t=self.extract_ticket(r)
    log.debug('Save Telescope (ticket %s)', t)
    r=self.s.post(u,data={'ticket':t,
                            'action':'part2-save',
                            'submittype':'Save',
                            'newTelescopeSelection':tele})
    t=self.extract_ticket(r)
    log.debug('GoTo Part 3 (ticket %s)', t)
    r=self.s.post(u,data={'ticket':t,'action':'main-go-part3'})
    t=self.extract_ticket(r)
    log.debug('Save Exposure (ticket %s)', t)
    r=self.s.post(u,data={'ticket':t,
                            'action':'part3-save',
                            'submittype':'Save',
                            'newExposureTime':exposure,
                            'newDarkFrame': 1 if darkframe else 0,
                            'newFilterSelection':filt,
                            'newRequestComments':comment})
    t=self.extract_ticket(r)
    log.debug('Submit (ticket %s)', t)
    r=self.s.post(u,data={'ticket':t, 'action':'main-submit'})
    return r

### Typical variable star job submission

In [30]:
#| export
@patch
def submitVarStar(self: Telescope, name, expos=90, filt='BVR',comm='', tele='COAST'):
    o=SkyCoord.from_name(name)
    return self.submit_job_api(o, name=name, comment=comm,
                            exposure=expos*1000, filt=filt, tele=tele)

In [31]:
#| local
if False :
    print("Submitting a VS job")
    rq = OSO.submitVarStar('V1223 Sgr', expos=180)
    if rq[0] :
        print("Waiting for job to be accepted")
        while (status:=OSO.get_request(int(rq[1]))['status'])!='Waiting' :
            print(status, end='\r')
            sys.stdout.flush()
            time.sleep(15)
        print(status)
        print("Cancelling the job")
        OSO.do_rm_api("0-cancel-request", {'rid':int(rq[1])})
        print("Waiting for job to be cancelled")
        while 'pending cancel' in (status:=OSO.get_request(int(rq[1]))['status']):
            print(status, end='\r')
            sys.stdout.flush()
            time.sleep(15)
        print(status)
    else :
        print('Submission failed')

In [32]:
#| local
reqlst=OSO.get_user_requests(sort='completion')
for rq in sorted(reqlst, key=lambda r: int(r['requesttime']), reverse=True)[:20]:
    print(f"{rq['objectname']:12}", 
          f"{datetime.datetime.fromtimestamp(int(rq['requesttime']))}",
          f"{rq['id']:12}",
          f"{Telescope.REQUESTSTATUS_TEXTS[int(rq['status'])]}"
         )

V1223 Sgr    2021-08-30 03:10:58 725917       Waiting
V1223 Sgr    2021-08-29 19:13:52 725886       Cancelled
V1223 Sgr    2021-08-28 18:43:05 725790       Cancelled
V1223 Sgr    2021-08-28 18:11:15 725783       Cancelled
V1223 Sgr    2021-08-28 18:03:06 725781       Cancelled
V1223 Sgr    2021-08-28 17:51:38 725780       Cancelled
V1223 Sgr    2021-08-28 14:51:45 725762       Cancelled
EQ Lyr       2021-08-26 15:37:56 725595       Waiting
DQ Vul       2021-08-26 15:37:54 725594       Complete
DX Vul       2021-08-26 15:37:52 725593       Complete
BI Her       2021-08-26 15:37:51 725592       Complete
AS Lac       2021-08-26 15:37:49 725591       Complete
V686 Cyg     2021-08-26 15:37:47 725590       Waiting
IP Cyg       2021-08-26 15:37:46 725589       Waiting
EU Cyg       2021-08-26 15:37:44 725588       Cancelled
SS Cyg       2021-08-26 15:37:42 725587       Complete
CH Cyg       2021-08-26 15:37:41 725586       Waiting
BI Her       2021-08-18 21:34:34 724987       Complete
EQ Lyr  

In [33]:
#| local
OSO.logout()

In [34]:
from nbdev import nbdev_export; nbdev_export()

Converted 00_core.ipynb.
Converted 01_solver.ipynb.
Converted 02_process.ipynb.
Converted index.ipynb.
